# 🌸 AIみちくさチャンネル 無料配布 Flux_1_Kontext_Dev版_起動 ノートブック 🌸

# 🍀 はじめに
- ＡＩみちくさチャンネル視聴者の皆様いつも本当にありがとうございます！🐣  
- このノートブックは、最新の ComfyUI を Google Drive に設置し、Flux_1_Kontext_Dev をどなたでも簡単に起動できる様にしたものです。


## 📄 目次
1. Google Drive のマウント  
2. ComfyUI 本体の取得と更新  
3. フォルダ構成の自動生成  
4. 必要モデルのダウンロード  
5. 依存関係のインストール  
6. cloudflared の準備  
7. ComfyUI の起動  

#🌎＜Step❶＞ 環境設定＆ComfyUI取得

In [ ]:
# セル①：Google Driveマウント → ComfyUIクローン／更新 → 依存関係インストール
from google.colab import drive
import subprocess, os

print("🌎 Google Drive をマウント中…")
drive.mount('/content/drive')

ROOT = '/content/drive/MyDrive'
WORKSPACE = os.path.join(ROOT, 'ComfyUI')

if not os.path.isdir(os.path.join(WORKSPACE, '.git')):
    print("📥 ComfyUI を初回クローン中…")
    subprocess.run(['git','clone','--depth','1',
                    'https://github.com/comfyanonymous/ComfyUI.git',
                    WORKSPACE], check=True)
else:
    print("🔄 ComfyUI を最新に更新中…")
    subprocess.run(['git','-C',WORKSPACE,'pull'], check=True)

print("📦 依存ライブラリをインストール中…（☕少しお待ちください※約3分）")
subprocess.run([
    'pip','install','-q',
    'xformers!=0.0.18',
    '-r', os.path.join(WORKSPACE,'requirements.txt'),
    '--extra-index-url','https://download.pytorch.org/whl/cu121'
], check=True)

print("✅ ＜Step❶＞セットアップ完了！次は📁＜Step❷＞を ▶️ 実行してください🌸")


#📁 ＜Step❷＞ フォルダ生成＆必要モデルダウンロード

In [ ]:
%%bash
# セル②：モデルフォルダを作成＆必要なモデルだけDL
WORKSPACE="/content/drive/MyDrive/ComfyUI"

echo "📁 フォルダ構成を作成中…"
mkdir -p "$WORKSPACE/models/diffusion_models" \
         "$WORKSPACE/models/vae" \
         "$WORKSPACE/models/text_encoders" \
         "$WORKSPACE/output"

echo "🌱 diffusion model をダウンロード…"
wget -q -c \
  https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors \
  -P "$WORKSPACE/models/diffusion_models/"

echo "🌼 VAE をダウンロード…"
wget -q -c \
  https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors \
  -P "$WORKSPACE/models/vae/"

echo "📖 text_encoders をダウンロード…"
wget -q -c \
  https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors \
  -P "$WORKSPACE/models/text_encoders/"
wget -q -c \
  https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors \
  -P "$WORKSPACE/models/text_encoders/"

echo "✅ 必要モデルの準備完了！"
echo "✅ ＜Step❷＞ セットアップ完了！次は💻＜Step❸＞を ▶️ 実行してください🌸"

#💻＜Step❸＞ ComfyUI 起動＆外部アクセスリンク表示

In [ ]:
# セル③：ComfyUI 起動＆Colab プロキシ経由リンク表示（URL重複を修正）
import subprocess, time, socket
from google.colab import output

WORKSPACE = '/content/drive/MyDrive/ComfyUI'

# 1️⃣ ComfyUI をバックグラウンド起動
print("🔧 ComfyUI を起動しています…")
subprocess.Popen(
    ['python','main.py','--listen','0.0.0.0','--port','8188'],
    cwd=WORKSPACE
)

# 2️⃣ ポート8188の立ち上がりを待機
print("⏳ ポート 8188 の起動を待っています…_☕少しお待ちください※約２分")
while True:
    time.sleep(0.5)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        if s.connect_ex(('127.0.0.1', 8188)) == 0:
            break
print("✅ ComfyUI 起動完了！")

# 3️⃣ Colab プロキシで公開URLを取得（既に https:// を含む）
url = output.eval_js("google.colab.kernel.proxyPort(8188)")

# 4️⃣ URLをそのまま表示
print(f"\n✅ 👇下記、ComfyUI アクセス用リンクをクリックして下さい🍀:\n{url}\n")
